In [9]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the CSV file into a DataFrame
df = pd.read_csv('Base_ScoreCredito_QuantumFinance.csv', delimiter=';')

# Display the first 5 rows
df.head()


,id,idade,sexo,estado_civil,escola,Qte_dependentes,tempo_ultimoservico,trabalha,vl_salario_mil,reg_moradia,casa_propria,vl_imovel_em_mil,Qte_cartoes,Qte_carros,SCORE_CREDITO
0,708082083,45,F,casado,graduacao,3,40,1,"40,089665024322",3,0,0,1,1,778
1,708083283,58,M,solteiro,ensino fundam,0,44,1,"66,557645274838",3,0,0,1,0,"276,3"
2,708084558,46,M,divorciado,doutorado,3,35,1,"123,68182089138",6,0,0,1,1,401
3,708085458,34,F,solteiro,mestrado,0,22,1,"19,7159343168329",6,0,0,1,0,"347,4"
4,708086958,49,F,casado,mestrado,2,36,0,0,3,1,437,1,2,476


In [11]:
# Get all unique values from `estado_civil`
estado_civil_unique_values = df['estado_civil'].unique()

# Get all unique values from `escola`
escola_unique_values = df['escola'].unique()

# Check the number of unique values in `estado_civil`
if len(estado_civil_unique_values) > 20:
  # If there are too many unique values, sample the top 20
  top_estado_civil_occurring_values = df['estado_civil'].value_counts().head(20).index.tolist()
  print(top_estado_civil_occurring_values)
else:
  # Otherwise print all unique valus in `estado_civil`
  print(estado_civil_unique_values)

# Check the number of unique values in `escola`
if len(escola_unique_values) > 20:
  # If there are too many unique values, sample the top 20
  top_escola_occurring_values = df['escola'].value_counts().head(20).index.tolist()
  print(top_escola_occurring_values)
else:
  # Otherwise print all unique valus in `escola`
  print(escola_unique_values)

['casado' 'solteiro' 'divorciado' 'na']
['graduacao' 'ensino fundam' 'doutorado' 'mestrado' 'ensino medio']


In [13]:
# Create a copy of the dataframe
df_encoded = df.copy()

# Convert `estado_civil` and `escola` to categorical data type
df_encoded['estado_civil'] = df_encoded['estado_civil'].astype('category')
df_encoded['escola'] = df_encoded['escola'].astype('category')

# One-hot encode the `estado_civil` and `escola` columns
df_encoded = pd.get_dummies(df_encoded, columns=['estado_civil', 'escola'])

# Convert the column `vl_salario_mil` to numeric
df_encoded['vl_salario_mil'] = df_encoded['vl_salario_mil'].astype(str).str.replace(',', '.')
df_encoded['vl_salario_mil'] = pd.to_numeric(df_encoded['vl_salario_mil'])

# Standardize the numerical features
for column in df_encoded.select_dtypes(include=['int64', 'float64']):
    if column != 'SCORE_CREDITO':  # Exclude the target variable from standardization
        df_encoded[column] = (df_encoded[column] - df_encoded[column].mean()) / df_encoded[column].std()

# Display the first 5 rows
print(df_encoded.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df_encoded.info())

| id        | idade      | sexo   | Qte_dependentes   | tempo_ultimoservico   | trabalha   | vl_salario_mil   | reg_moradia   | casa_propria   | vl_imovel_em_mil   | Qte_cartoes   | Qte_carros   | SCORE_CREDITO   | estado_civil_casado   | estado_civil_divorciado   | estado_civil_na   | estado_civil_solteiro   | escola_doutorado   | escola_ensino fundam   | escola_ensino medio   | escola_graduacao   | escola_mestrado   |
|:----------|:-----------|:-------|:------------------|:----------------------|:-----------|:-----------------|:--------------|:---------------|:-------------------|:--------------|:-------------|:----------------|:----------------------|:--------------------------|:------------------|:------------------------|:-------------------|:-----------------------|:----------------------|:-------------------|:------------------|
| -0.842611 | -0.165397  | F      | 0.97547           | 0.713598              | 0.310772   | -0.542036        | -0.495702     | -0.737677      | -0.5521

In [15]:
print(df_encoded.isnull().sum().to_markdown(numalign="left", stralign="left"))

|                         | 0   |
|:------------------------|:----|
| id                      | 0   |
| idade                   | 0   |
| sexo                    | 0   |
| Qte_dependentes         | 0   |
| tempo_ultimoservico     | 0   |
| trabalha                | 0   |
| vl_salario_mil          | 0   |
| reg_moradia             | 0   |
| casa_propria            | 0   |
| vl_imovel_em_mil        | 0   |
| Qte_cartoes             | 0   |
| Qte_carros              | 0   |
| SCORE_CREDITO           | 0   |
| estado_civil_casado     | 0   |
| estado_civil_divorciado | 0   |
| estado_civil_na         | 0   |
| estado_civil_solteiro   | 0   |
| escola_doutorado        | 0   |
| escola_ensino fundam    | 0   |
| escola_ensino medio     | 0   |
| escola_graduacao        | 0   |
| escola_mestrado         | 0   |


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Split the data into features and target
X = df_encoded.drop('SCORE_CREDITO', axis=1)
y = df_encoded['SCORE_CREDITO']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the Linear Regression model on the train set
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the test set using R-squared and RMSE metrics
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'R-squared: {r2}')
print(f'RMSE: {rmse}')

ValueError: could not convert string to float: 'F'

In [19]:
# Drop the column `id`
df_encoded = df_encoded.drop('id', axis=1)

# Convert the column `sexo` to numeric
df_encoded['sexo'] = df_encoded['sexo'].replace({'M': 1, 'F': 0})

# Display the first 5 rows
print(df_encoded.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df_encoded.info())

| idade      | sexo   | Qte_dependentes   | tempo_ultimoservico   | trabalha   | vl_salario_mil   | reg_moradia   | casa_propria   | vl_imovel_em_mil   | Qte_cartoes   | Qte_carros   | SCORE_CREDITO   | estado_civil_casado   | estado_civil_divorciado   | estado_civil_na   | estado_civil_solteiro   | escola_doutorado   | escola_ensino fundam   | escola_ensino medio   | escola_graduacao   | escola_mestrado   |
|:-----------|:-------|:------------------|:----------------------|:-----------|:-----------------|:--------------|:---------------|:-------------------|:--------------|:-------------|:----------------|:----------------------|:--------------------------|:------------------|:------------------------|:-------------------|:-----------------------|:----------------------|:-------------------|:------------------|
| -0.165397  | 0      | 0.97547           | 0.713598              | 0.310772   | -0.542036        | -0.495702     | -0.737677      | -0.552184          | -0.250574     | 0.6609

C:\Users\rafae\AppData\Local\Temp\ipykernel_5848\1340608897.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded['sexo'] = df_encoded['sexo'].replace({'M': 1, 'F': 0})


In [21]:
# Split the data into features and target
X = df_encoded.drop('SCORE_CREDITO', axis=1)
y = df_encoded['SCORE_CREDITO']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the Linear Regression model on the train set
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the test set using R-squared and RMSE metrics
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'R-squared: {r2}')
print(f'RMSE: {rmse}')

ValueError: could not convert string to float: '389,7'

In [23]:
# Convert the column `SCORE_CREDITO` to numeric
df_encoded['SCORE_CREDITO'] = df_encoded['SCORE_CREDITO'].astype(str).str.replace(',', '.')
df_encoded['SCORE_CREDITO'] = pd.to_numeric(df_encoded['SCORE_CREDITO'])

# Display the first 5 rows
print(df_encoded.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df_encoded.info())

| idade      | sexo   | Qte_dependentes   | tempo_ultimoservico   | trabalha   | vl_salario_mil   | reg_moradia   | casa_propria   | vl_imovel_em_mil   | Qte_cartoes   | Qte_carros   | SCORE_CREDITO   | estado_civil_casado   | estado_civil_divorciado   | estado_civil_na   | estado_civil_solteiro   | escola_doutorado   | escola_ensino fundam   | escola_ensino medio   | escola_graduacao   | escola_mestrado   |
|:-----------|:-------|:------------------|:----------------------|:-----------|:-----------------|:--------------|:---------------|:-------------------|:--------------|:-------------|:----------------|:----------------------|:--------------------------|:------------------|:------------------------|:-------------------|:-----------------------|:----------------------|:-------------------|:------------------|
| -0.165397  | 0      | 0.97547           | 0.713598              | 0.310772   | -0.542036        | -0.495702     | -0.737677      | -0.552184          | -0.250574     | 0.6609

In [25]:
# Split the data into features and target
X = df_encoded.drop('SCORE_CREDITO', axis=1)
y = df_encoded['SCORE_CREDITO']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the Linear Regression model on the train set
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the test set using R-squared and RMSE metrics
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'R-squared: {r2}')
print(f'RMSE: {rmse}')

R-squared: 0.6200483648985325
RMSE: 81.596134845782


C:\Users\rafae\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
